<a href="https://colab.research.google.com/github/ebceran/fraud-detection--in-progress-/blob/main/Fraud_Detection_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
df = pd.read_csv("creditcard.csv")
print("Shape:", df.shape)
print(df.dtypes.head())
print("NA total:", df.isna().sum().sum())
print(df["Class"].value_counts())
df.head()

Shape: (19898, 31)
Time      int64
V1      float64
V2      float64
V3      float64
V4      float64
dtype: object
NA total: 20
Class
0.0    19812
1.0       85
Name: count, dtype: int64


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [4]:
# Which columns contain null values?
print(df.isna().sum()[df.isna().sum() > 0])

V11       1
V12       1
V13       1
V14       1
V15       1
V16       1
V17       1
V18       1
V19       1
V20       1
V21       1
V22       1
V23       1
V24       1
V25       1
V26       1
V27       1
V28       1
Amount    1
Class     1
dtype: int64


In [6]:
# Delete the null contain columns?
df = df.dropna()
print("New shape:", df.shape)

New shape: (19897, 31)


In [7]:
print(df["Class"].value_counts())
df["Class"].value_counts(normalize=True)

Class
0.0    19812
1.0       85
Name: count, dtype: int64


,proportion
Class,
0.0,0.995728
1.0,0.004272


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# X and y
X = df.drop("Class", axis=1)
y = df["Class"]

# Training / test place
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Guess
y_pred = model.predict(X_test)

# Results
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred, digits=4))

[[5936    8]
 [   9   17]]
              precision    recall  f1-score   support

         0.0     0.9985    0.9987    0.9986      5944
         1.0     0.6800    0.6538    0.6667        26

    accuracy                         0.9972      5970
   macro avg     0.8392    0.8263    0.8326      5970
weighted avg     0.9971    0.9972    0.9971      5970



/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
import numpy as np

# X, y zaten tanımlı
X = df.drop("Class", axis=1)
y = df["Class"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("logreg", LogisticRegression(
        max_iter=5000,
        class_weight="balanced",
        solver="liblinear"
    ))
])

pipe.fit(X_train, y_train)

# 0.5 eşik
y_pred = pipe.predict(X_test)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, digits=4))

# --- Eşik oynaması (örnek: 0.3, 0.5, 0.7) ---
proba = pipe.predict_proba(X_test)[:,1]
for th in [0.3, 0.5, 0.7]:
    y_hat = (proba >= th).astype(int)
    p, r, f1, _ = precision_recall_fscore_support(y_test, y_hat, average="binary", zero_division=0)
    tp = np.sum((y_test==1)&(y_hat==1))
    fp = np.sum((y_test==0)&(y_hat==1))
    fn = np.sum((y_test==1)&(y_hat==0))
    print(f"\nThreshold={th:.1f} → Precision={p:.3f}  Recall={r:.3f}  F1={f1:.3f} | TP={tp}, FP={fp}, FN={fn}")


Confusion Matrix:
 [[5880   64]
 [   4   22]]

Classification Report:
               precision    recall  f1-score   support

         0.0     0.9993    0.9892    0.9943      5944
         1.0     0.2558    0.8462    0.3929        26

    accuracy                         0.9886      5970
   macro avg     0.6276    0.9177    0.6936      5970
weighted avg     0.9961    0.9886    0.9916      5970


Threshold=0.3 → Precision=0.200  Recall=0.846  F1=0.324 | TP=22, FP=88, FN=4

Threshold=0.5 → Precision=0.256  Recall=0.846  F1=0.393 | TP=22, FP=64, FN=4

Threshold=0.7 → Precision=0.375  Recall=0.808  F1=0.512 | TP=21, FP=35, FN=5
